# SPRINT - 2

## Item - Levantamento dos cálculos dos indicadores.

## Item - Levantamento do procedimento de consolidação dos dados de produção e apontamentos.

## Item - Desenvolvimento do processamento de Limpeza, Formatação e Classificação dos dados, e gravação na "Refined Data".

<br>
<br>

Autor.: Sérgio C. Medina

#### Declaração dos Pacotes, Libs ou Classes utilizadas no processo.

In [1]:
# Declaração dos Pacotes, Libs ou Classes utilizadas no processo.
import os
import io
import pandas as pd
from google.cloud import storage


# configurando variavel de ambiente com o arquivo de credenciais para conexão GCP
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "..\..\secrets\edc-igti-smedina-4920e12ac565.json"

# Conexão com o Cloud Storage e criação do objeto bucket referente a área de dados
storage_client = storage.Client()
bucket = storage_client.get_bucket("edc-pa-i4-data")

In [7]:
# Função para leitura do arquivo CSV no GCS convertando para Pandas Dataframe
def gcp_csv_to_df(folder, dtexec, lineprod=None, sep=";"):

    # formata o padrão do nome do arquivo
    file_name = None
    if (dtexec is not None and lineprod is None):
        file_name = f"{dtexec}.csv"
    elif (dtexec is not None and lineprod is not None):
        file_name = f"{lineprod}-{dtexec}.csv"
    else:
        raise Exception("Invalid parameters.")

    # define o objeto blob com o path para o arquivo no GCS
    blob = bucket.blob(f"raw-data-zone/{folder}/{file_name}")

    # Faz o download no formato em bytes e posiciona no index 0
    byte_stream = io.BytesIO()
    blob.download_to_file(byte_stream)
    byte_stream.seek(0)

    # Converte para o Dataframe em Pandas
    df = pd.read_csv(byte_stream, sep=sep)

    return df



#### Leituda de dados na "raw-data-zone" pasta "DATAOP"

In [11]:
dtexec = "2021-11-08"

df_dataop = gcp_csv_to_df(folder="dataop", dtexec=dtexec, sep=";")
df_dataprod = gcp_csv_to_df(folder="dataprod", dtexec=dtexec, lineprod="101", sep=";")


In [13]:
df_dataprod

,TIMESTAMP,LINE,OP,BATCH,TIMER,TOTMIN,STSID,STSDS,PC,GOOD,REJECT
0,2021-11-08 00:00:00,101,0,0,00:00:00,0,0,STOP,0,0,0
1,2021-11-08 00:04:48,101,0,0,00:04:48,"4,8",0,STOP,0,0,0
2,2021-11-08 00:12:58,101,0,0,00:08:10,"8,17",0,STOP,0,0,0
3,2021-11-08 00:18:24,101,0,0,00:05:26,"5,43",0,STOP,0,0,0
4,2021-11-08 00:24:42,101,0,0,00:06:18,"6,3",0,STOP,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
193,2021-11-08 23:33:04,101,211108L101,TB70394,00:13:57,"13,95",4,WORKING,14,14,0
194,2021-11-08 23:38:20,101,211108L101,TB70394,00:05:16,"5,27",4,WORKING,5,5,0
195,2021-11-08 23:47:05,101,211108L101,TB70394,00:08:45,"8,75",4,WORKING,9,9,0
196,2021-11-08 23:55:13,101,211108L101,TB70394,00:08:08,"8,13",4,WORKING,8,8,0
